# Installations

Install and unzip TIMIT dataset from google drive

In [1]:
# ! gdown 15Nq4PdOY7h8AP54ge3EurUPKa91jvezc

In [2]:
# ! unzip timit.zip -d ./timit/

In [3]:
import os
import random
import json
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import IPython.display as ipd
import librosa
import soundfile as sf
import datasets

import torch
import torch.nn as nn
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor
from jiwer import wer


from typing import List, Tuple, Set, Dict
%matplotlib inline

/home/artur/files/Audio Processing/phoneme_recognition/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

This section contains a modified version of the original function, adapted from [here](https://www.kaggle.com/code/vitouphy/phoneme-recognition-with-wav2vec2/notebook) and [here](https://huggingface.co/blog/fine-tune-wav2vec2-english) to fit our specific task requirements.

In [4]:
timit_path = "./timit/"
timit_data_path = "./timit/data/"

In [5]:
df_train = pd.read_csv(os.path.join(timit_path, 'train_data.csv'))
df_test = pd.read_csv(os.path.join(timit_path, 'test_data.csv'))
df = pd.concat([df_train, df_test])
df = df[df['is_converted_audio'] == False]

In [6]:
df_train = df[df['test_or_train'] == "TRAIN"]
df_test  = df[df['test_or_train'] == "TEST"]

In [7]:
df_train

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file
1,2.0,TRAIN,DR4,MMDM0,SI1311.PHN,TRAIN/DR4/MMDM0/SI1311.PHN,TRAIN\\DR4\\MMDM0\\SI1311.PHN,False,False,False,True,False
2,3.0,TRAIN,DR4,MMDM0,SI1311.WRD,TRAIN/DR4/MMDM0/SI1311.WRD,TRAIN\\DR4\\MMDM0\\SI1311.WRD,False,False,True,False,False
3,4.0,TRAIN,DR4,MMDM0,SX321.PHN,TRAIN/DR4/MMDM0/SX321.PHN,TRAIN\\DR4\\MMDM0\\SX321.PHN,False,False,False,True,False
4,5.0,TRAIN,DR4,MMDM0,SX321.WRD,TRAIN/DR4/MMDM0/SX321.WRD,TRAIN\\DR4\\MMDM0\\SX321.WRD,False,False,True,False,False
5,6.0,TRAIN,DR4,MMDM0,SI681.TXT,TRAIN/DR4/MMDM0/SI681.TXT,TRAIN\\DR4\\MMDM0\\SI681.TXT,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
23093,23094.0,TRAIN,DR8,MRDM0,SX245.TXT,TRAIN/DR8/MRDM0/SX245.TXT,TRAIN\\DR8\\MRDM0\\SX245.TXT,False,False,False,NaN,True
23094,23095.0,TRAIN,DR8,MRDM0,SI1044.PHN,TRAIN/DR8/MRDM0/SI1044.PHN,TRAIN\\DR8\\MRDM0\\SI1044.PHN,False,False,False,NaN,False
23096,23097.0,TRAIN,DR8,MRDM0,SX245.WAV,TRAIN/DR8/MRDM0/SX245.WAV,TRAIN\\DR8\\MRDM0\\SX245.WAV,False,True,False,NaN,False
23098,23099.0,TRAIN,DR8,MRDM0,SX335.WAV,TRAIN/DR8/MRDM0/SX335.WAV,TRAIN\\DR8\\MRDM0\\SX335.WAV,False,True,False,NaN,False


In [8]:
df_test

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file
0,1.0,TEST,DR4,MGMM0,SX139.WAV,TEST/DR4/MGMM0/SX139.WAV,TEST\\DR4\\MGMM0\\SX139.WAV,False,True,False,False,False
2,3.0,TEST,DR4,MGMM0,SX139.TXT,TEST/DR4/MGMM0/SX139.TXT,TEST\\DR4\\MGMM0\\SX139.TXT,False,False,False,False,True
3,4.0,TEST,DR4,MGMM0,SI499.WRD,TEST/DR4/MGMM0/SI499.WRD,TEST\\DR4\\MGMM0\\SI499.WRD,False,False,True,False,False
4,5.0,TEST,DR4,MGMM0,SX319.WRD,TEST/DR4/MGMM0/SX319.WRD,TEST\\DR4\\MGMM0\\SX319.WRD,False,False,True,False,False
5,6.0,TEST,DR4,MGMM0,SX319.PHN,TEST/DR4/MGMM0/SX319.PHN,TEST\\DR4\\MGMM0\\SX319.PHN,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8394,8395.0,TEST,DR8,MPAM0,SX19.WAV,TEST/DR8/MPAM0/SX19.WAV,TEST\\DR8\\MPAM0\\SX19.WAV,False,True,False,False,False
8395,8396.0,TEST,DR8,MPAM0,SX109.TXT,TEST/DR8/MPAM0/SX109.TXT,TEST\\DR8\\MPAM0\\SX109.TXT,False,False,False,False,True
8397,8398.0,TEST,DR8,MPAM0,SX289.WRD,TEST/DR8/MPAM0/SX289.WRD,TEST\\DR8\\MPAM0\\SX289.WRD,False,False,True,False,False
8398,8399.0,TEST,DR8,MPAM0,SX109.WAV,TEST/DR8/MPAM0/SX109.WAV,TEST\\DR8\\MPAM0\\SX109.WAV,False,True,False,False,False


In this part, we will process the raw data to structure the dataset into three columns:

1. Path to the audio file
2. Text from the audio
3. Corresponding phonemes

Additionally, we will filter out any samples lacking phoneme data, as they cannot be used for training or testing without labels.

In [9]:
PHONETIC_KEY: str = "phonetic"
AUDIO_KEY:    str = "audio"
WORD_KEY:     str = "word"


def convert_phon61_to_phon39(sentence: str) -> str:
    phon61_map39 = {
        'iy':'iy',  'ih':'ih',  'eh':'eh',  'ae':'ae',   'ix':'ih',  'ax':'ah',  'ah':'ah',   'uw':'uw',
        'ux':'uw',  'uh':'uh',  'ao':'aa',  'aa':'aa',   'ey':'ey',  'ay':'ay',  'oy':'oy',   'aw':'aw',
        'ow':'ow',  'l':'l',    'el':'l',   'r':'r',     'y':'y',    'w':'w',    'er':'er',   'axr':'er',
        'm':'m',    'em':'m',   'n':'n',    'nx':'n',    'en':'n',   'ng':'ng',  'eng':'ng',  'ch':'ch',
        'jh':'jh',  'dh':'dh',  'b':'b',    'd':'d',     'dx':'dx',  'g':'g',    'p':'p',     't':'t',
        'k':'k',    'z':'z',    'zh':'sh',  'v':'v',     'f':'f',    'th':'th',  's':'s',     'sh':'sh',
        'hh':'hh',  'hv':'hh',  'pcl':'h#', 'tcl':'h#',  'kcl':'h#', 'qcl':'h#', 'bcl':'h#',  'dcl':'h#',
        'gcl':'h#', 'h#':'h#',  '#h':'h#',  'pau':'h#',  'epi':'h#', 'nx':'n',   'ax-h':'ah', 'q':'h#'
    }
    tokens = [phon61_map39[x] for x in sentence.split()]
    return " ".join(tokens)


def read_text_file(filepath: str) -> str:
    with open(filepath) as f:
        tokens = [line.split()[-1] for line in f]
        return " ".join(tokens)


def merge_item_data(df: pd.DataFrame) -> Dict[str, Dict[str, str]]:
    data = {}

    for _, row in tqdm(df.iterrows()):
        path = row['path_from_data_dir']
        entry_id = path.split('.')[0]
        entry_id = "-".join(entry_id.split('/')[-2:])

        if entry_id not in data:
            data[entry_id] = {}

        if row['is_audio'] is True:
            data[entry_id][AUDIO_KEY] = os.path.join(timit_data_path, path)
        elif row['is_word_file'] is True:
            data[entry_id][WORD_KEY] = read_text_file(os.path.join(timit_data_path, path))
        elif row['is_phonetic_file'] is True:
            phoneme_sentence = read_text_file(os.path.join(timit_data_path, path))
            data[entry_id][PHONETIC_KEY] = convert_phon61_to_phon39(phoneme_sentence)
    return data


def transform_dataset(df: pd.DataFrame) -> Dict[str, List[str]]:
    merged_data = merge_item_data(df)

    phonetic_files: List[str] = []
    audio_files:    List[str] = []
    word_files:     List[str] = []
    keys_set:       Set[str]  = set([PHONETIC_KEY, AUDIO_KEY, WORD_KEY])

    for _, value in merged_data.items():
        if set(value) == keys_set:
            phonetic_files.append(value[PHONETIC_KEY])
            audio_files.append(value[AUDIO_KEY])
            word_files.append(value[WORD_KEY])

    return {
        AUDIO_KEY: audio_files,
        WORD_KEY: word_files,
        PHONETIC_KEY: phonetic_files
    }

In [10]:
train_val = transform_dataset(df_train)
test = transform_dataset(df_test)

18480it [00:01, 13227.18it/s]
6720it [00:00, 12526.60it/s]


In [11]:
random.seed(42)
n = len(train_val[AUDIO_KEY])
train_n = int(n * 0.9)
val_n = n - train_n

val_ind = np.array(sorted(random.sample(range(0, n), val_n)))
train_ind = np.array(sorted(list(set(range(0, n)) - set(val_ind))))

In [12]:
train = dict()
val = dict()

for key in [AUDIO_KEY, WORD_KEY, PHONETIC_KEY]:
    train[key] = np.array(train_val[key])[train_ind].tolist()
    val[key]   = np.array(train_val[key])[val_ind].tolist()

In [13]:
print(f"Train set length: {len(train[AUDIO_KEY])}")
print(f"Validation set length: {len(val[AUDIO_KEY])}")
print(f"Test set length: {len(test[AUDIO_KEY])}")

Train set length: 1512
Validation set length: 168
Test set length: 1680


Now let's convert the data to a data type that the model will work with.

In [14]:
train_dataset = datasets.Dataset.from_dict(train)
val_dataset   = datasets.Dataset.from_dict(val)
test_dataset  = datasets.Dataset.from_dict(test)

In [15]:
train_dataset = train_dataset.cast_column(AUDIO_KEY, datasets.Audio(sampling_rate=16_000))
val_dataset   = val_dataset.cast_column(AUDIO_KEY, datasets.Audio(sampling_rate=16_000))
test_dataset  = test_dataset.cast_column(AUDIO_KEY, datasets.Audio(sampling_rate=16_000))

In [16]:
ind = 10

print("Text:", train_dataset[ind][WORD_KEY])
print("Phonetics:", train_dataset[ind][PHONETIC_KEY])
print("Input array shape:", train_dataset[ind][AUDIO_KEY]["array"].shape)
print("Sampling rate:", train_dataset[ind][AUDIO_KEY]["sampling_rate"])
ipd.Audio(data=train_dataset[ind][AUDIO_KEY]["array"], autoplay=False, rate=16000)

Text: get a calico cat to keep
Phonetics: h# g ih dx ih h# k ae l ih h# k ow h# k ae h# t ah h# k iy h# p h#
Input array shape: (32359,)
Sampling rate: 16000


#### Phonemes vocabulary

In [17]:
train_phonetics = [phone for x in train_dataset for phone in x[PHONETIC_KEY].split()]
val_phonetics   = [phone for x in val_dataset for phone in x[PHONETIC_KEY].split()]
test_phonetics  = [phone for x in test_dataset for phone in x[PHONETIC_KEY].split()]

In [18]:
vocab_list = list(set(train_phonetics) | set(val_phonetics) | set(test_phonetics) | set([' ']))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
print(vocab_dict)

{' ': 0, 'aa': 1, 'ae': 2, 'ah': 3, 'aw': 4, 'ay': 5, 'b': 6, 'ch': 7, 'd': 8, 'dh': 9, 'dx': 10, 'eh': 11, 'er': 12, 'ey': 13, 'f': 14, 'g': 15, 'h#': 16, 'hh': 17, 'ih': 18, 'iy': 19, 'jh': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'ng': 25, 'ow': 26, 'oy': 27, 'p': 28, 'r': 29, 's': 30, 'sh': 31, 't': 32, 'th': 33, 'uh': 34, 'uw': 35, 'v': 36, 'w': 37, 'y': 38, 'z': 39}


In [19]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

42


In [20]:
with open('./vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
del df
del df_test
del df_train

# Modeling

### Wav2Vec2 

In [22]:
DEVICE = "cuda"

ssl_model_name = "facebook/wav2vec2-base-960h"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(ssl_model_name)
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

ssl_model = Wav2Vec2Model.from_pretrained(ssl_model_name, output_hidden_states=True).to(DEVICE)
ssl_model.eval()

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [23]:
ind: int = 0
inputs = processor(train_dataset['audio'][ind]['array'], 
                   sampling_rate=train_dataset['audio'][ind]['sampling_rate'], 
                   return_tensors="pt", 
                   padding=True).input_values.to(DEVICE)

with torch.no_grad():
    outputs = ssl_model(inputs)

hidden_states = outputs.hidden_states
for i, hs in enumerate(hidden_states):
    print(f"Layer {i}: {hs.shape}")

Layer 0: torch.Size([1, 270, 768])
Layer 1: torch.Size([1, 270, 768])
Layer 2: torch.Size([1, 270, 768])
Layer 3: torch.Size([1, 270, 768])
Layer 4: torch.Size([1, 270, 768])
Layer 5: torch.Size([1, 270, 768])
Layer 6: torch.Size([1, 270, 768])
Layer 7: torch.Size([1, 270, 768])
Layer 8: torch.Size([1, 270, 768])
Layer 9: torch.Size([1, 270, 768])
Layer 10: torch.Size([1, 270, 768])
Layer 11: torch.Size([1, 270, 768])
Layer 12: torch.Size([1, 270, 768])


In [24]:
from torch.utils.data import DataLoader, Dataset

class PhonemeDataset(Dataset):
    def __init__(self, data: datasets.Dataset, layer: int, save_extra_data: bool = False):
        self.data = []
        self.extra_data = []

        print("Preprocessing data...")
        for audio, phonetic in tqdm(zip(data[AUDIO_KEY], data[PHONETIC_KEY])):
            input = processor(audio["array"], 
                              sampling_rate=audio["sampling_rate"], 
                              return_tensors="pt", 
                              padding=True).input_values.to(DEVICE)
            with torch.no_grad():
                outputs = ssl_model(input)
            features = outputs.hidden_states[-layer].cpu()

            with processor.as_target_processor():
                labels = processor(phonetic).input_ids

            self.data.append({
                'features': features[0],
                'labels': torch.tensor(labels)
            })

        if save_extra_data:
            print("Saving extra data...")
            for audio, words, phonetic in tqdm(zip(data[AUDIO_KEY], data[WORD_KEY], data[PHONETIC_KEY])):
                self.extra_data.append({
                    AUDIO_KEY: audio,
                    WORD_KEY: words,
                    PHONETIC_KEY: phonetic
                })

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        return self.data[idx]


    def get_extra_data(self, idx: int) -> Dict:
        if len(self.extra_data) == 0:
            raise ValueError("Extra data is missing")
        return self.extra_data[idx]

In [25]:
train_dataset = PhonemeDataset(train_dataset, 1)
val_dataset   = PhonemeDataset(val_dataset, 1)
test_dataset  = PhonemeDataset(test_dataset, 1, save_extra_data=True)

Preprocessing data...


0it [00:00, ?it/s]/home/artur/files/Audio Processing/phoneme_recognition/venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
1512it [00:34, 43.96it/s]


Preprocessing data...


168it [00:03, 44.22it/s]


Preprocessing data...


1680it [00:38, 43.89it/s]


Saving extra data...


1680it [00:00, 1039756.64it/s]


### Heads

In [26]:
class LinearHead(nn.Module):
    def __init__(self, input_dim: int, output_dim: int):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)


    def forward(self, x):
        return self.fc(x)



class MLPHead(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, dropout_prob: float, output_dim: int):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        return x

### Utils

In [36]:
def data_collator(batch: List[Dict[str, torch.tensor]]):    
    features = [sample['features'] for sample in batch]
    labels   = [sample['labels'] for sample in batch]
    padded_features = torch.nn.utils.rnn.pad_sequence(features, batch_first=True)

    labels_length = torch.tensor([len(label) for label in labels], dtype=torch.long)
    labels = torch.cat([label.clone().detach().long() for label in labels])
    return padded_features, labels, labels_length

In [51]:
def predict_phonemes(model: nn.Module, features: torch.Tensor, reversed_vocab_dict: Dict[int, str], ctc_blank: int) -> str:
    features = features.to(DEVICE)

    with torch.no_grad():
        logits = model(features).cpu()

    probs = torch.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1).squeeze().numpy()

    merged = [preds[0]]
    for pred in preds:
        if pred != merged[-1]:
            merged.append(pred)
    
    result = []
    for val in merged:
        if val != ctc_blank and val in reversed_vocab_dict:
            result.append(reversed_vocab_dict[val])

    return " ".join(result)


def show_results(model: nn.Module, dataset: PhonemeDataset, idx: int, reversed_vocab_dict: Dict[int, str], ctc_blank: int) -> List[float]:
    data = dataset.get_extra_data(idx)
    text = data[WORD_KEY]
    phonetics = data[PHONETIC_KEY]
    audio = data[AUDIO_KEY]["array"]
    
    features, _, _ = data_collator([dataset[idx]])
    features = features.to(DEVICE)
    prediction = predict_phonemes(model, features, reversed_vocab_dict, ctc_blank)
    
    print(f"Text: {text}")
    print(f"PER: {wer(reference=phonetics, hypothesis=prediction)}")
    print(f"Predicted phonetics:\n{phonetics}")
    print(f"Ground Truth phonetics:\n{prediction}")
    return audio

In [29]:
def train_epoch(model: nn.Module, train_loader: DataLoader, criterion: nn.Module, optimizer: torch.optim, disable_tqdm: bool = False) -> float:
    train_loss = 0.0

    model.train()
    for features, labels, labels_lengths in tqdm(train_loader, disable=disable_tqdm):
        features = features.to(DEVICE)
        labels = labels.to(DEVICE)
        labels_lengths = labels_lengths.to(DEVICE)

        optimizer.zero_grad()
        logits = model(features)
        log_probs = torch.log_softmax(logits, dim=-1).permute(1, 0, 2) #(sequence, batch, classes)

        input_lengths = torch.full(
            size=(logits.size(0),), 
            fill_value=logits.size(1), 
            dtype=torch.long,
            device=DEVICE
        )
        
        loss = criterion(log_probs, labels, input_lengths, labels_lengths)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    return train_loss / len(train_loader)


def evaluate_model(model: nn.Module, val_loader: DataLoader, reversed_vocab_dict: Dict, ctc_blank: int, disable_tqdm: bool = False) -> float:
    total_per = 0.0

    model.eval()
    for features, labels, _ in tqdm(val_loader, disable=disable_tqdm):
        expected = []
        for val in labels.numpy():
            if val in reversed_vocab_dict:
                expected.append(reversed_vocab_dict[val])
        ground_truth = " ".join(expected)

        predicted = predict_phonemes(model, features, reversed_vocab_dict, ctc_blank)
        total_per += wer(ground_truth, predicted)
    return total_per / len(val_loader)


def train_model(model: nn.Module, 
                train_loader: DataLoader, 
                val_loader: DataLoader, 
                criterion: nn.Module, 
                optimizer: torch.optim, 
                reversed_vocab_dict: Dict, 
                ctc_blank: int,
                weights_path: str,
                disable_tqdm: bool = False) -> Tuple[float, float]:

    num_epochs = 10
    best_per = 1

    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion=criterion, optimizer=optimizer, disable_tqdm=disable_tqdm)
        val_per = evaluate_model(model, val_loader, reversed_vocab_dict, ctc_blank, disable_tqdm=disable_tqdm)
        if best_per - val_per > 0.001:
            torch.save(model.state_dict(), weights_path)
            best_per = val_per
        if not disable_tqdm:
            print(f"Epoch [{epoch + 1}/{num_epochs}], "
                f"Train Loss: {train_loss}, "
                f"Validation PER: {val_per:.4f}")
    return train_loss, best_per

In [30]:
reversed_vocab_dict = dict()

for key in vocab_dict:
    reversed_vocab_dict[vocab_dict[key]] = key 

### Training

In [62]:
classes = len(vocab_dict) + 1
input_dim = 768
batch_size = 1
ctc_blank = len(vocab_dict)

model = LinearHead(input_dim=input_dim, 
                   output_dim=classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
ctc_loss = nn.CTCLoss(blank=ctc_blank)
train_loader = DataLoader(train_dataset, collate_fn=data_collator, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, collate_fn=data_collator, batch_size=1)
test_loader  = DataLoader(test_dataset, collate_fn=data_collator, batch_size=1)

In [63]:
train_loss, best_val_per = train_model(model=model, 
                                       train_loader=train_loader, 
                                       val_loader=val_loader, 
                                       criterion=ctc_loss, 
                                       optimizer=optimizer, 
                                       reversed_vocab_dict=reversed_vocab_dict, 
                                       ctc_blank=ctc_blank, 
                                       weights_path="./weights/wav2vec_linear.pt")

test_per = evaluate_model(model=model,
                          val_loader=test_loader,
                          reversed_vocab_dict=reversed_vocab_dict,
                          ctc_blank=ctc_blank)

print(f"\nFinal loss: {train_loss}")
print(f"Best validation PER: {best_val_per}")
print(f"Test PER: {test_per}")

100%|██████████| 168/168 [00:00<00:00, 1703.91it/s]


Epoch [1/10], Train Loss: 3.5496140514258987, Validation PER: 0.5720


100%|██████████| 168/168 [00:00<00:00, 1497.12it/s]


Epoch [2/10], Train Loss: 1.7236472851542568, Validation PER: 0.5125


100%|██████████| 168/168 [00:00<00:00, 1707.30it/s]


...

Epoch [9/10], Train Loss: 1.104016210154566, Validation PER: 0.3927


100%|██████████| 168/168 [00:00<00:00, 1724.29it/s]


Epoch [10/10], Train Loss: 1.0804952429164023, Validation PER: 0.3838


100%|██████████| 1680/1680 [00:01<00:00, 1635.10it/s]



Final loss: 1.0804952429164023
Best validation PER: 0.38379900886525936
Test PER: 0.38651708514237


In [65]:
ipd.Audio(data=show_results(model, test_dataset, 10, reversed_vocab_dict, 42), autoplay=False, rate=16000)

Text: this brochure is particularly informative for prospective buyer
PER: 0.48936170212765956
Predicted phonetics:
h# dh ih s h# b r ow sh er ih s h# p ah h# t ih h# k l ih n f ow m h# t ih f er h# p er s h# p ah h# t ih v h# b ay er h#
Ground Truth phonetics:
dh ih s h# p r sh er s p dx ih h# k ih n f aa er m v f ah er p er s p h# ih v b aa


In [ ]:
# model = MLPHead(input_dim=input_dim, 
#                 hidden_dim=512,
#                 dropout_prob=0.5,
#                 output_dim=classes).to(DEVICE)